In [1]:
%%javascript
//FOR MAKING EVERYTHING TABS IN JUYPYTER NOTEBOOK, LEAVE HERE FOR NOW
//ALSO MAKE SURE YOU *ALWAYS* RUN THIS WHENEVER YOU OPEN THE NOTEBOOK
//BEFORE EVERY DOING ANYTHING
IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [2]:
max_iter = 100

#turn this off for handin, whether the model is verbose or not
verbose_on = True

In [3]:
import sys, codecs, json, math, time, warnings
warnings.simplefilter( action='ignore', category=FutureWarning )

import nltk, scipy, sklearn, sklearn_crfsuite, sklearn_crfsuite.metrics, eli5
nltk.download('averaged_perceptron_tagger')
from sklearn.metrics import make_scorer
from collections import Counter
import matplotlib.pyplot as plt
from IPython.display import display    
import numpy as np

import logging
import tensorflow as tf
import absl.logging
formatter = logging.Formatter('[%(levelname)s|%(filename)s:%(lineno)s %(asctime)s] %(message)s')
absl.logging.get_absl_handler().setFormatter(formatter)
absl.logging._warn_preinit_stderr = False
logger = tf.get_logger()
logger.setLevel(logging.INFO)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
#Takes the whole book that has been read as a string and generates
#the dictionary needed containing the table of contents of the book
def task1(book_str):
    
	contents = r"(CONTENTS|(C|c)ontents)"
	
	#either a roman numeral or digit
	number = r"([ivxlcdm]+|[IVXLCDM]+|\d+)"
	
	#the charcters which have been seen to appear in titles I've looked at
	chset = r"[A-Za-z0-9\?\-\.\'\":“”’]"

In [5]:
#Takes the chapter that has been read as a string and extracts all 
#questions from it into the required set
def task2(chapter_str):
    
    #Anything can be part of a question
    #starts with a capital letter followed by any number of 
    # "non-ending" characters
	question_regex = r"[A-Z][^\?\.,!]*\?"
    
    #doesnt really need to be match objects, maybe change this later
	return set([match_obj[0] for match_obj in re.finditer(question_regex, chapter_str, flags = re.MULTILINE | re.DOTALL)])

In [6]:
#Tasks 3 and 4 are sort of linked so MAYBE handling all their initialisation
#stuff make more sense within a single function? Not sure, leave here for now

#def init_NER(chapter_str):

In [7]:
#Takes the chapter that has been read as a string and finds the 
#NER entitites listed in the task 3 spec, generating a dictionary
#containing these that the names found in task 4 can then be 
#injected into
def task3(chapter_str):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-7-da00bff9fe22>, line 6)

In [ ]:
#Takes the chapter that has been read as a string and finds the names
#of all characters, returning a list that can be put into the task 3/4
#dictionary writer
def task4(chapter_str):
    

In [ ]:
#SUBTASK 1, takes the whole book file as an argument

#all you have to do is populate dictTOC as shown commented out and 
#the function will handle writing everything for you etc.
def exec_regex_toc( file_book = None ) :

	book_str = ""
	with open(file_book, 'r') as f:
		book_str = f.read()
    
	dictTOC = task1(book_str)
        
        
	# INSERT CODE TO USE REGEX TO BUILD A TABLE OF CONTENTS FOR A BOOK (subtask 1)

#	# hardcoded output to show exactly what is expected to be serialized
#	dictTOC = {
#			"1": "I AM BORN",
#			"2": "I OBSERVE",
#			"3": "I HAVE A CHANGE"
#		}

	return dictTOC

	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	"""
	writeHandle = codecs.open( 'toc.json', 'w', 'utf-8', errors = 'replace' )
	strJSON = json.dumps( dictTOC, indent=2 )
	writeHandle.write( strJSON + '\n' )
	writeHandle.close()
	"""

In [ ]:
#SUBTASK 2, take the file chapter as argument and handles writing the list of questions

#all you ahve to do is populate setQuestions as is shown commented out
#and the function will then handle writing everything and stuff for you
def exec_regex_questions( file_chapter = None ) :

	chapter_str = ""
	with open(file_chapter, 'r') as f:
		chapter_str = f.read()
    
	setQuestions = task2(chapter_str)
    
	# INSERT CODE TO USE REGEX TO LIST ALL QUESTIONS IN THE CHAPTER OF TEXT (subtask 2)

	# hardcoded output to show exactly what is expected to be serialized
#	setQuestions = set([
#		"Traddles?",
#		"And another shilling or so in biscuits, and another in fruit, eh?",
#		"Perhaps you’d like to spend a couple of shillings or so, in a bottle of currant wine by and by, up in the bedroom?",
#		"Has that fellow’--to the man with the wooden leg--‘been here again?"
#		])

	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	return setQuestions
	
	"""
	writeHandle = codecs.open( 'questions.txt', 'w', 'utf-8', errors = 'replace' )
	for strQuestion in setQuestions :
		writeHandle.write( strQuestion + '\n' )
	writeHandle.close()
	"""

In [16]:
from sklearn.model_selection import ShuffleSplit
from nltk.corpus import names
from nltk import pos_tag

In [17]:
#take the file path of ontonotes and return sentences
def load_ontonotes(dataset_file):

	readHandle = codecs.open(dataset_file, 'r', 'utf-8', errors = 'replace')
	str_json = readHandle.read()
	readHandle.close()
	dict_ontonotes = json.loads(str_json)
	orig_list = list(dict_ontonotes.keys())

	#the counter is just for testing, remove it later maybe
	i = 0

	#the sentences of triplets containing the tokens, nltk POS tags
	#and NER tags from ontonotes
	sentences = []

	for str_file in orig_list:
	#REMOVE THIS ITS FOR TESTING
		#print("on file number:", i)
		i += 1
		for str_sent_index in dict_ontonotes[str_file] :

			tokens = []
			ner_tags = []
			# compute IOB tags for named entities (if any)
			ne_type_last = None

			#build up the list of tokens and ner tags
			for nTokenIndex in range(len(dict_ontonotes[str_file][str_sent_index]['tokens'])) :
				strToken = dict_ontonotes[str_file][str_sent_index]['tokens'][nTokenIndex]
				strPOS = dict_ontonotes[str_file][str_sent_index]['pos'][nTokenIndex]
				ne_type = None
				if 'ne' in dict_ontonotes[str_file][str_sent_index] :
					dict_ne = dict_ontonotes[str_file][str_sent_index]['ne']
					if not 'parse_error' in dict_ne :
						for str_NEIndex in dict_ne :
							if nTokenIndex in dict_ne[str_NEIndex]['tokens'] :
								ne_type = dict_ne[str_NEIndex]['type']
								break
				if ne_type != None :
					if ne_type == ne_type_last :
						strIOB = 'I-' + ne_type
					else :
						strIOB = 'B-' + ne_type
				else :
					strIOB = 'O'

				ne_type_last = ne_type
				tokens.append(strToken)
				ner_tags.append(strIOB)
				#list_entry.append( ( strToken, strIOB ) )

			#this should probably be removed, leave it here for now though
			"""
			skip = True

			for tag in ner_tags:
				#display label subset contains the NER tags that we are looking for
				if tag in display_label_subset:
					skip = False
					break

			#this sentence does not contain any of the named entity tags
			#that we are looking for so it is useless, don't bother using it
			#if skip:
				#continue
			"""

			list_entry = []

			#use nltk pos tags instead
			for (index, tup) in enumerate(pos_tag(tokens)):
				#print("tup is:", tup)
				list_entry.append((tup[0], tup[1], ner_tags[index]))
			sentences.append( list_entry )
	return sentences

In [18]:
from sklearn.model_selection import ShuffleSplit
from nltk.corpus import names
from copy import copy
from random import shuffle
nltk.download("names")

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [19]:
#given the loaded ontonotes sentences and the sentences from the chapter
#which is having entities extracted from it, return the final NER dictionary for tasks 3 and 4
def run_NER(sentences, sentences_to_predict):

	#given a specific NER tag, return 2 sets consisting of the 
	#"inside parts" and the "before parts"
	def extract_gazetteers(target, sentences):
		before = set()
		inside = set()
		before_tag = "B-" + target
		inside_tag = "I-" + target
		for sent in sentences:
			for tok, _, ner in sent:
				if ner == before_tag:
					before.add(tok.lower())
				if ner == inside_tag:
					inside.add(tok.lower())
		return before, inside


	#convert all the NER tags you don't care about into O's in sentences that may have mixed NER tags that we care about
	#with NER tags that we don't care about
	def only_Os(desired_set, sentences):
		actual_tags = []
		for tag in desired_set:
			actual_tags.append("B-" + tag)
			actual_tags.append("I-" + tag)
		return [[(tok, pos, ner if ner in actual_tags else 'O') for tok, pos, ner in sent] for sent in sentences]


	#If the dataset has already been split and we KNOW that none of these NER tags are the ones we care about,
	#then clearly we can just replace ALL NER tags with O
	def all_Os(sentences):
		return [[(tok, pos, 'O') for tok, pos, _ in sent] for sent in sentences]


	#returns a tuples of 2 lists of sentences, where one list contains all the sentences that contain the tags 
	#specified in the argument and the other list does not
	#you just give it the tag strings and it adds the Bs and Is for you
	def split_tags(tag_set, sentences):

		actual_tags = []
		for tag in tag_set:
			actual_tags.append("B-" + tag)
			actual_tags.append("I-" + tag)

		contains_tags = []
		doesnt_contain_tags = []

		for sent in sentences:

			if(any([tag in actual_tags for _,_,tag in sent])):
				contains_tags.append(sent)
			else:
				doesnt_contain_tags.append(sent)

		return (contains_tags, doesnt_contain_tags)

	#the sentences containing task 3 tags and the sentences that dont
	wtask3, wotask3 = split_tags(["DATE", "CARDINAL", "ORDINAL", "NORP"], sentences)

	#the sentences containing task 4 tags (only person) and the sentences that dont
	wtask4, wotask4 = split_tags(["PERSON"], sentences)

	#all other tags can be converted to O because we dont care about finding them
	task3_processed = only_Os(["DATE", "CARDINAL", "ORDINAL", "NORP"], wtask3)
	task4_processed = only_Os(["PERSON"], wtask4)

	
	#take a random sample with the specified size from the list of sentences
	def random_sample(sentences, num):
		cp = copy(sentences)
		shuffle(cp)
		return cp[-num:]
	
	#split the items into training and testing denoted by the testing ratio argument
	def random_split(items, ratio):
		shuffle(copy(items))
		train_size = int(ratio * len(items))
		test_size = len(items) - train_size
		return items[:train_size], items[-test_size:]

	#don't need training and testing for handin, just use the whole thing
	#task3_training, task3_testing = random_split(task3_processed, 0.9)
	#task4_training, task4_testing = random_split(task4_processed, 0.9)

	#generate the gazetteers to be used from ontonotes
	#before_name, inside_name = extract_gazetteers("PERSON", wtask4)
	#before_date, inside_date = extract_gazetteers("DATE", wtask3)
	#before_cardinal, inside_cardinal = extract_gazetteers("CARDINAL", wtask3)
	#before_norp, inside_norp = extract_gazetteers("NORP", wtask3)
	#before_ord, inside_ord = extract_gazetteers("ORDINAL", wtask3)

		#assumed to be in the environment of the function that 
	#generates the feature dictionary for task 3
	number_gazetteer = set([
		"one", "two", "three", "four", "five", "six", "seven", "eight",
		"nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
		"sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "fourty", "fifty",
		"sixty", "seventy", "eighty", "ninety", "hundred", "onehundred", "one-hundred", "thousand", "million"
		])

	#Python sets are implemented as hash tables so lookup is always around O(1)
	#even for large sets

	ordinal_gazetteer = set([
		"first", "second", "third", "fourth", "fifth", "sixth", "seventh",
		"eigth", "ninth", "tenth", "eleventh", "twelth", "thirteenth",
		"fourteenth", "fifteenth", "sixteenth", "seventeenth", "eighteenth",
		"nineteenth", "twentieth", "thirtieth", "fourtieth", "fiftieth", 
		"sixtieth", "seventieth", "eightieth", "ninetieth", "hundreth", "thousandth",
		"millionth", "firstly", "secondly", "thirdly", "fourthly", "fifthly", "sixthly",
		"seventhly", "eigthly", "ninethly", "tenthly"
	])

	date_gazetteer = set([
		"monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "day", "week", "month", "year"
		"jan", "january", "feb", "february", "mar", "march", "apr", "april", "may", "jun", "june", "jul", "july",
		"aug", "august", "sep", "september", "oct", "october", "nov", "november", "dec", "december", "holiday", "holidays"
	])

	time_gazetteer = set(["second", "seconds", "minute", "minutes", "hour", "hours", "time", "clock", "o'clock", "past", "oclock"])

	# a feature for detecting e.g. 1st, 2nd, 3rd, 4th etc.
	def is_st_th(candidate):
		suffixes = set(["st", "nd", "rd", "th"])
		return candidate[-2:] in suffixes and candidate[:-2].isdigit()

	#use the nltk name corpus
	name_gazetteer = set(map(lambda name: name.lower(), names.words("male.txt") + names.words("female.txt")))

	#task 4 has less features as it only revolves around names
	def gen_feature_dict_task4(sentence, i):

		def gen_features(index, relative_str):
			return {
				relative_str + "word.lower()": sentence[index][0].lower(),
				relative_str + "word_prefix": sentence[index][0][:3],
				relative_str + "word_suffix": sentence[index][0][-3:],
				relative_str + "postag": sentence[index][1],
		  #relative_str + "postag_prefix2": sentence[index][1][:2], THINK THESE HARM THE MODEL
				#relative_str + "postag_prefix3": sentence[index][1][:3], #think this is useless?
				relative_str + "istitle": sentence[index][0].istitle(),
				relative_str + "isupper": sentence[index][0].isupper()     
			}

		#extra features are limited to center 3 in the context window of size 5
		def gen_extra_features(index, relative_str):
			return {
				relative_str + "contains_digit" : any(char.isdigit() for char in sentence[index][0]),
				relative_str + "in_nltk_name_gaz" :  sentence[index][0].lower() in name_gazetteer,
#				relative_str + "in_B_name" : sentence[index][0].lower() in before_name,
#				relative_str + "in_I_name" : sentence[index][0].lower() in inside_name
			}

		features = gen_features(i, "")
		features.update(gen_extra_features(i, ""))

		#at least 1 word behind
		if i > 0:#generate a list of feature dictionaries for each word in the sentence
			features.update(gen_features(i - 1, "-1:"))
			features.update(gen_extra_features(i - 1, "-1:"))

		#at least 2 words behind
		if i > 1:
			features.update(gen_features(i - 2, "-2:"))

		#at least 1 word ahead
		if i < len(sentence) - 1:
			features.update(gen_features(i + 1, "+1:"))
			features.update(gen_extra_features(i + 1, "+1:"))

		#at least 2 words ahead
		if i < len(sentence) - 2:
			features.update(gen_features(i + 2, "+2:"))

		return features

	def gen_feature_dict_task3(sentence, i):

		#token = sentence[i][0]
		#pos_tag = sentence[i][1]

		def gen_features(index, relative_str):

			return {
				relative_str + "word.lower()": sentence[index][0].lower(),
				relative_str + "word_prefix": sentence[index][0][:3],
				relative_str + "word_suffix": sentence[index][0][-3:],
				relative_str + "postag": sentence[index][1],
		  #relative_str + "postag_prefix2": sentence[index][1][:2], THINK THESE HARM THE MODEL
				#relative_str + "postag_prefix3": sentence[index][1][:3], #think this is useless?
				relative_str + "istitle": sentence[index][0].istitle(),
				relative_str + "isupper": sentence[index][0].isupper(),        
			}

		#extra features are limited to center 3 in the context window of size 5
		def gen_extra_features(index, relative_str):
			return {
				relative_str + "contains_digit": any(char.isdigit() for char in sentence[index][0]), 
				relative_str + "contains_day": "day" in sentence[index][0].lower(),

				#python sets are around O(1) lookup so its fine to have large gazetteers
				relative_str + "in_num_gaz": sentence[index][0].lower() in number_gazetteer,
				relative_str + "in_ord_gaz": sentence[index][0].lower() in ordinal_gazetteer,
				relative_str + "in_date_gaz": sentence[index][0].lower() in date_gazetteer,
				relative_str + "in_time_gaz": sentence[index][0].lower() in time_gazetteer,    

				#COMMENT THESE OUT MAYBE COS NOT SURE ABOUT THEM
#				relative_str + "in_B_date": sentence[index][0].lower() in before_date,
#				relative_str + "in_I_date": sentence[index][0].lower() in inside_date,
#				relative_str + "in_B_card": sentence[index][0].lower() in before_cardinal,
#				relative_str + "in_I_card": sentence[index][0].lower() in inside_cardinal,
#				relative_str + "in_B_norp": sentence[index][0].lower() in before_norp,
#				relative_str + "in_I_norp": sentence[index][0].lower() in inside_norp,
#				relative_str + "in_B_ord": sentence[index][0].lower() in before_ord,
#				relative_str + "in_I_ord": sentence[index][0].lower() in inside_ord,
				relative_str + "is_st_th": is_st_th(sentence[index][0])
			}

		features = gen_features(i, "")
		features.update(gen_extra_features(i, ""))

		#there is at least 1 word behind
		if i > 0:
			features.update(gen_features(i - 1, "-1:"))
			features.update(gen_extra_features(i - 1, "-1:"))

		#there are at least 2 words behind
		if i > 1:
			features.update(gen_features(i - 2, "-2:"))

		#there is at least 1 word ahead
		if i < len(sentence) - 1:
			features.update(gen_features(i + 1, "+1:"))
			features.update(gen_extra_features(i + 1, "+1:"))

		#there are at least 2 words ahead
		if i < len(sentence) - 2:
			features.update(gen_features(i + 2, "+2:"))

		return features

		#generate a list of feature dictionaries for each word in the sentence
	def gen_task3_features(sentence):
		return [gen_feature_dict_task3(sentence, i) for i in range(len(sentence))]

	def gen_task4_features(sentence):
		return [gen_feature_dict_task4(sentence, i) for i in range(len(sentence))]

	#generate lists of feature dictionaries for each sentence
	def task3(ontonotes_data, sentences_to_predict):
		feature_lists = [gen_task3_features(sent) for sent in sentences]
		print("Made the task 3 feature lists")
		label_lists = [sent2labels(sent) for sent in sentences]
		crf = sklearn_crfsuite.CRF(
			algorithm = "lbfgs",
			c1 = 1.587763,
			c2 = 0.149853957,
			max_iterations = max_iter,
			all_possible_transitions = True,
			verbose = verbose_on
		)
		crf.fit(feature_lists, label_lists)
		predictions = crf.predict(sentences_to_predict)
		return extract_entities_from_sequences(
			["CARDINAL", "ORDINAL", "DATE", "NORP"], predictions, True)

	#try seeing if different hyperparamaters for each model improves
	#their respsective performances, might be worth them not both being the same
	def task4(ontonotes_data, sentences_to_predict):
		feature_lists = [gen_task4_features(sent) for sent in ontonotes_data]
		print("Made the task 4 feature lists")
		label_lists = [sent2labels(sent) for sent in ontonotes_data]
		crf = sklearn_crfsuite.CRF(
			algorithm = "lbfgs",
			c1 = 1.587763,
			c2 = 0.149853957,
			max_iterations = max_iter,
			all_possible_transitions = True,
			verbose = verbose_on
		)
		crf.fit(feature_lists, label_lists)
		predictions = crf.predict(sentences_to_predict)
		return extract_entities_from_sequences(["PERSON"], predictions, False)

	#Think this is all that my compie can handle :)
	task3_processed = random_sample(task3_processed, 10000)
	task4_processed = random_sample(task4_processed, 10000)
	
	print("TASK 3 SIZE:")
	print(len(task3_processed))
	
	print("TASK 4 SIZE:")
	print(len(task4_processed))
	
	ner_dict = task3(task3_processed, sentences_to_predict)
	ner_dict.update(task4(task4_processed, sentences_to_predict))

	#phewwww finally :)
	return ner_dict

In [20]:
from nltk import sent_tokenize, word_tokenize

In [21]:
#Used for taking the chapter file and ontonotes dataset as an argument
#then training the task 3 and task 4 models and writing the dictionary
#results for each of them

#all you have to do is fill up dictNE as is shown commented out and
#the function handles writing everything for you
def exec_ner( file_chapter = None, ontonotes_file = None ) :

	#First read the whole chapter in as a string
	chapter_str = ""
	with open(file_chapter, 'r') as f:
		chapter_str = f.read()
	
	sentences_to_predict = []
	for sent in sent_tokenize(chapter_str):
		sentences_to_predict.append(word_tokenize(sent))
	
	ontonotes_sentences = load_ontonotes(ontonotes_file)
	dictNE = run_NER(ontonotes_sentences, sentences_to_predict)
	
	# INSERT CODE TO TRAIN A CRF NER MODEL TO TAG THE CHAPTER OF TEXT (subtask 3)
	# USING NER MODEL AND REGEX GENERATE A SET OF BOOK CHARACTERS AND FILTERED SET OF NE TAGS (subtask 4)

	# hardcoded output to show exactly what is expected to be serialized (you should change this)
#	dictNE = {
#			"CARDINAL": [
#				"two",
#				"three",
#				"one"
#			],
#			"ORDINAL": [
#				"first"
#			],
#			"DATE": [
	#			"saturday",
#			],
#			"NORP": [
#				"indians"
#			],
#			"PERSON": [
#				"creakle",
#				"mr. creakle",
#				"mrs. creakle",
#				"miss creakle"
#			]
#		}

	return dictNE

"""
	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	# write out all PERSON entries for character list for subtask 4
	writeHandle = codecs.open( 'characters.txt', 'w', 'utf-8', errors = 'replace' )
	if 'PERSON' in dictNE :
		for strNE in dictNE['PERSON'] :
			writeHandle.write( strNE.strip().lower()+ '\n' )
	writeHandle.close()

	# FILTER NE dict by types required for subtask 3
	listAllowedTypes = [ 'DATE', 'CARDINAL', 'ORDINAL', 'NORP' ]
	listKeys = list( dictNE.keys() )
	for strKey in listKeys :
		for nIndex in range(len(dictNE[strKey])) :
			dictNE[strKey][nIndex] = dictNE[strKey][nIndex].strip().lower()
		if not strKey in listAllowedTypes :
			del dictNE[strKey]

	# write filtered NE dict
	writeHandle = codecs.open( 'ne.json', 'w', 'utf-8', errors = 'replace' )
	strJSON = json.dumps( dictNE, indent=2 )
	writeHandle.write( strJSON + '\n' )
	writeHandle.close()
	"""

"\n\t# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK\n\n\t# write out all PERSON entries for character list for subtask 4\n\twriteHandle = codecs.open( 'characters.txt', 'w', 'utf-8', errors = 'replace' )\n\tif 'PERSON' in dictNE :\n\t\tfor strNE in dictNE['PERSON'] :\n\t\t\twriteHandle.write( strNE.strip().lower()+ '\n' )\n\twriteHandle.close()\n\n\t# FILTER NE dict by types required for subtask 3\n\tlistAllowedTypes = [ 'DATE', 'CARDINAL', 'ORDINAL', 'NORP' ]\n\tlistKeys = list( dictNE.keys() )\n\tfor strKey in listKeys :\n\t\tfor nIndex in range(len(dictNE[strKey])) :\n\t\t\tdictNE[strKey][nIndex] = dictNE[strKey][nIndex].strip().lower()\n\t\tif not strKey in listAllowedTypes :\n\t\t\tdel dictNE[strKey]\n\n\t# write filtered NE dict\n\twriteHandle = codecs.open( 'ne.json', 'w', 'utf-8', errors = 'replace' )\n\tstrJSON = json.dumps( dictNE, indent=2 )\n\twriteHandle.write( strJSON + '\n' )\n\twriteHandle.close()\n\t"

In [22]:
#test each of the subtasks showing all the dictionary results
#use eval_book and eval_chapter
def do_testing():
    
	#task1dict = exec_regex_toc("eval_book.txt")
	print("---TASK 1 DICTIONARY RESUTLS---")
	#print(task1dict)
	#task2question_set = exec_regex_questions("eval_chapter.txt")
	print("---TASK 2 QUESTIONS FOUND---")
	#print(task2question_set)
	task3and4_NER_dict = exec_ner("eval_chapter.txt", "ontonotes_parsed.json")
	print("---TASK 3 and 4 DICTIONARY FOUND---")
	print(task3and4_NER_dict)

In [ ]:
do_testing()

---TASK 1 DICTIONARY RESUTLS---
---TASK 2 QUESTIONS FOUND---
TASK 3 SIZE:
10000
TASK 4 SIZE:
10000
